In [3]:
!pip install openrouteservice geopy


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import openrouteservice
from geopy.geocoders import Nominatim
from datetime import datetime, date, timedelta

ORS_API_KEY = "5b3ce3597851110001cf624838ed54901e044a6aa4d688103f2b9208"
client = openrouteservice.Client(key=ORS_API_KEY)
geolocator = Nominatim(user_agent="meeting-app")
today = date.today()

def get_coordinates(place):
    location = geolocator.geocode(place, timeout=10)
    if location:
        return (location.longitude, location.latitude)
    else:
        raise Exception(f"Invalid location: {place}")

def get_travel_time(user_coords, meeting_coords, mode="driving-car"):
    route = client.directions(
        coordinates=[user_coords, meeting_coords],
        profile=mode,
        format='json'
    )
    duration_sec = route['routes'][0]['summary']['duration']
    return duration_sec / 60  # return in minutes

while True:
    try:
        name = input("Enter user name: ")
        current_time_input = input("Enter current time (e.g. 03:00 PM): ")
        meeting_time_input = input("Enter meeting time (e.g. 03:30 PM): ")
        user_place = input("Enter user location: ")
        meeting_place = input("Enter meeting location: ")

        current_time = datetime.combine(today, datetime.strptime(current_time_input, "%I:%M %p").time())
        meeting_time = datetime.combine(today, datetime.strptime(meeting_time_input, "%I:%M %p").time())

        user_coords = get_coordinates(user_place)
        meeting_coords = get_coordinates(meeting_place)

        travel_minutes = get_travel_time(user_coords, meeting_coords)
        arrival_time = current_time + timedelta(minutes=travel_minutes)

        if arrival_time > meeting_time:
            delay_minutes = round((arrival_time - meeting_time).total_seconds() / 60)
            print(f"{name}: delay ({delay_minutes} minutes late)")
        else:
            print(f"{name}: on time")

    except Exception as e:
        print(f"Error: {e}")

    again = input("Add another user? (yes/no): ").strip().lower()
    if again != "yes":
        break

Enter user name:  samah
Enter current time (e.g. 03:00 PM):  03:00 PM
Enter meeting time (e.g. 03:30 PM):  03:15 PM
Enter user location:  Diarb Negm
Enter meeting location:  zagazig


samah: delay (5 minutes late)


Add another user? (yes/no):  yes
Enter user name:  samah
Enter current time (e.g. 03:00 PM):  03:00 PM
Enter meeting time (e.g. 03:30 PM):  03:15 PM
Enter user location:  03
Enter meeting location:  


Error: Invalid location: 


Add another user? (yes/no):  yes
Enter user name:  samah
Enter current time (e.g. 03:00 PM):  03:00 PM
Enter meeting time (e.g. 03:30 PM):  03:20 PM
Enter user location:  Diarb Negm
Enter meeting location:  zagazig University


samah: delay (2 minutes late)


Add another user? (yes/no):  yes
Enter user name:  samah
Enter current time (e.g. 03:00 PM):  03:00 PM
Enter meeting time (e.g. 03:30 PM):  03:30 PM
Enter user location:  Diarb Negm
Enter meeting location:  zagazig


samah: on time
